In [2]:
# import basic modules
import sys
import os
import time
import numpy as np
from tqdm import tqdm
import gc
import torch
import argparse
import skimage.transform

# import custom modules
code_dir = '/user_data/mmhender/imStat/code/'
sys.path.append(code_dir)
from feature_extraction import texture_statistics_gabor, texture_statistics_pyramid, bdcn_features, sketch_token_features
from utils import nsd_utils, roi_utils

bdcn_path = '/user_data/mmhender/toolboxes/BDCN/'

from model_fitting import initialize_fitting, merge_features, fwrf_fit, fwrf_predict

fpX = np.float32
device = 'cpu:0'
# device = initialize_fitting.init_cuda()

In [3]:
fitting_type='pyramid_texture'
do_avg_pool=True
subject=1
volume_space = True
up_to_sess = 1
n_ori = 4
n_sf = 4
nonlin_fn = False
padding_mode = 'circular';
group_all_hl_feats = True; \
sample_batch_size = 50; voxel_batch_size = 100; \
zscore_features = True; ridge = True; \
shuffle_images = False; random_images = False; random_voxel_data = False; \
do_fitting = True; do_val = True; do_varpart = True; date_str = None;
shuff_rnd_seed = 0; debug = True; \
do_pca = False; min_pct_var = 99; max_pc_to_retain = 400; map_ind = -1; \
n_prf_sd_out = 2; mult_patch_by_prf = True; \
downsample_factor = 1.0; do_nms = False

In [4]:
if 'pyramid' in fitting_type:
    model_name = initialize_fitting.get_pyramid_model_name(ridge, n_ori, n_sf)
    feature_types_exclude = []        
    name1 = 'pyramid_texture'

elif 'gabor_texture' in fitting_type:        
    model_name = initialize_fitting.get_gabor_texture_model_name(ridge, n_ori, n_sf)
    feature_types_exclude = []
    name1 = 'gabor_texture'

elif 'gabor_solo' in fitting_type:        
    model_name = initialize_fitting.get_gabor_solo_model_name(ridge, n_ori, n_sf)
    feature_types_exclude = ['pixel', 'simple_feature_means', 'autocorrs', 'crosscorrs']
    name1 = 'gabor_solo'

elif 'bdcn' in fitting_type:
    model_name = initialize_fitting.get_bdcn_model_name(do_pca, map_ind)   
    name1 = 'bdcn'

elif 'sketch_tokens' in fitting_type:
    model_name = initialize_fitting.get_sketch_tokens_model_name(do_pca)   
    name1 = 'sketch_tokens'

else:
    raise ValueError('your string for fitting_type was not recognized')

if 'plus_sketch_tokens' in fitting_type:
    model_name2 = initialize_fitting.get_sketch_tokens_model_name(do_pca)
    model_name = model_name + '_plus_' + model_name2
elif 'plus_bdcn' in fitting_type:
    model_name2 = initialize_fitting.get_bdcn_model_name(do_pca, map_ind)
    model_name = model_name + '_plus_' + model_name2


output_dir, fn2save = initialize_fitting.get_save_path(subject, volume_space, model_name, shuffle_images, random_images, random_voxel_data, debug, date_str)

# decide what voxels to use  
voxel_mask, voxel_index, voxel_roi, voxel_ncsnr, brain_nii_shape = roi_utils.get_voxel_roi_info(subject, volume_space)

sessions = np.arange(0,up_to_sess)
zscore_betas_within_sess = True
# get all data and corresponding images, in two splits. always fixed set that gets left out
trn_stim_data, trn_voxel_data, val_stim_data, val_voxel_data, \
        image_order, image_order_trn, image_order_val = nsd_utils.get_data_splits(subject, sessions=sessions, \
                                                                     voxel_mask=voxel_mask, volume_space=volume_space, \
                                                                      zscore_betas_within_sess=zscore_betas_within_sess, \
                                                                      shuffle_images=shuffle_images, random_images=random_images, \
                                                                                         random_voxel_data=random_voxel_data)


if 'gabor' in fitting_type or 'sketch_tokens' in fitting_type or 'pyramid' in fitting_type:
    # For this model, the features are pre-computed, so we will just load them rather than passing in images.
    # Going to pass the image indices (into 10,000 dim array) instead of images to fitting and val functions, 
    # which will tell which features to load.
    trn_stim_data = image_order_trn
    val_stim_data = image_order_val


Time Stamp: Sep-21-2021_2203_43

Will save final output file to /user_data/mmhender/imStat/model_fits/S01/texture_pyramid_ridge_4ori_4sf/Sep-21-2021_2203_43_DEBUG/


Volume space: ROI defs are located at: /lab_data/tarrlab/common/datasets/NSD/nsddata/ppdata/subj01/func1pt8mm/roi

3794 voxels of overlap between kastner and prf definitions, using prf defs
unique values in retino labels:
[-1.  0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16.
 17. 18. 19. 20. 21. 22. 23. 24. 25.]
0 voxels of overlap between face and place definitions, using place defs
unique values in categ labels:
[-1.  0. 26. 27. 28. 30. 31. 32. 33.]
1535 voxels are defined (differently) in both retinotopic areas and category areas

14913 voxels are defined across all areas, and will be used for analysis

Loading numerical label/name mappings for all ROIs:
[1, 2, 3, 4, 5, 6, 7]
['V1v', 'V1d', 'V2v', 'V2d', 'V3v', 'V3d', 'hV4']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 2

In [5]:
     
# More params for fitting
holdout_size, lambdas = initialize_fitting.get_fitting_pars(trn_voxel_data, zscore_features, ridge=ridge)
# Params for the spatial aspect of the model (possible pRFs)
aperture_rf_range = 1.1
aperture, models = initialize_fitting.get_prf_models(aperture_rf_range=aperture_rf_range)    
    


Possible lambda values are:
[1.0000000e+00 4.2169652e+00 1.7782795e+01 7.4989418e+01 3.1622775e+02
 1.3335215e+03 5.6234131e+03 2.3713736e+04 1.0000000e+05]
most extreme RF positions:
[-0.55 -0.55  0.04]
[0.55       0.55       0.40000001]


In [7]:
 
do_pca_pyr_hl = True
do_pca_st = True

group_all_hl_feats = True
feature_types_exclude =[]
# Set up the pyramid
compute_features = False
_fmaps_fn = steerable_pyramid_extractor(pyr_height = n_sf, n_ori = n_ori)
# Initialize the "texture" model which builds on first level feature maps
_feature_extractor1 = texture_feature_extractor(_fmaps_fn,sample_batch_size=sample_batch_size, \
                  subject=subject, feature_types_exclude=feature_types_exclude, n_prf_sd_out=n_prf_sd_out, \
                                               aperture=aperture, do_varpart = do_varpart, \
                              group_all_hl_feats = group_all_hl_feats, compute_features = compute_features, \
                               do_pca_hl=do_pca_pyr_hl, min_pct_var = min_pct_var, max_pc_to_retain = max_pc_to_retain, \
                               device=device)
# feature_info = [_feature_extractor.feature_column_labels, _feature_extractor.feature_types_include]
        

Feature types to exclude from the model:
[]


In [8]:
map_resolution = 227
_feature_extractor2 = sketch_token_features.sketch_token_feature_extractor(subject, device, map_resolution=map_resolution, \
                                                                           aperture = aperture, \
                                                     n_prf_sd_out = n_prf_sd_out, \
                               batch_size=sample_batch_size, mult_patch_by_prf=mult_patch_by_prf, do_avg_pool = do_avg_pool,\
                                           do_pca = do_pca_st, min_pct_var = min_pct_var, max_pc_to_retain = max_pc_to_retain)
_feature_extractor = merge_features.combined_feature_extractor([_feature_extractor1, _feature_extractor2], \
                                                                   [name1,'sketch_tokens'], do_varpart = do_varpart)


In [14]:
defined

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [9]:
mm=2
_feature_extractor1.init_for_fitting((240,240), models, dtype=np.float32)
feats1, defined1 = _feature_extractor1(val_stim_data, models[mm], mm, fitting_mode=True)

In [14]:

mm=2
_feature_extractor2.init_for_fitting((240,240), models, dtype=np.float32)
feats2, defined2 = _feature_extractor2(val_stim_data, models[mm], mm, fitting_mode=True)

Initializing for fitting
Initializing arrays for PCA params
Clearing features from memory
Loading pre-computed features from /user_data/mmhender/features/sketch_tokens/S1_features_each_prf.h5py
Took 19.29789 seconds to load file
Size of features array for this image set is:
(62, 151, 875)
Preparing for PCA: original dims of features:
(62, 151)
Running PCA...
Retaining 1 components to expl 99 pct var
Final size of feature matrix is:
(62, 1)


In [10]:
mm=2
_feature_extractor.init_for_fitting((240,240), models, dtype=np.float32)
feats, defined = _feature_extractor(val_stim_data, models[mm], mm, fitting_mode=True)

Initializing for fitting
Initializing arrays for PCA params
Clearing precomputed features from memory.
Initializing for fitting
Initializing arrays for PCA params
Clearing features from memory
Loading pre-computed features for models [0 - 49] from /user_data/mmhender/features/pyramid_texture/S1_features_each_prf_4ori_4sf.h5py
Took 95.55682 seconds to load file
Index into batch for prf 2: 2
Size of features array for this image set and prf is:
(62, 641)
Final size of features concatenated is [62 x 641]
Feature types included are:
['pixel_stats', 'mean_magnitudes', 'mean_realparts', 'marginal_stats_lowpass_recons', 'variance_highpass_resid', 'magnitude_feature_autocorrs', 'lowpass_recon_autocorrs', 'highpass_resid_autocorrs', 'magnitude_within_scale_crosscorrs', 'real_within_scale_crosscorrs', 'magnitude_across_scale_crosscorrs', 'real_imag_across_scale_crosscorrs', 'real_spatshift_within_scale_crosscorrs', 'real_spatshift_across_scale_crosscorrs']
Final size of features concatenated is 

In [15]:
feats, defined = _feature_extractor(val_stim_data, models[mm], mm, fitting_mode=False)

Index into batch for prf 2: 2
Size of features array for this image set and prf is:
(62, 641)
Final size of features concatenated is [62 x 641]
Feature types included are:
['pixel_stats', 'mean_magnitudes', 'mean_realparts', 'marginal_stats_lowpass_recons', 'variance_highpass_resid', 'magnitude_feature_autocorrs', 'lowpass_recon_autocorrs', 'highpass_resid_autocorrs', 'magnitude_within_scale_crosscorrs', 'real_within_scale_crosscorrs', 'magnitude_across_scale_crosscorrs', 'real_imag_across_scale_crosscorrs', 'real_spatshift_within_scale_crosscorrs', 'real_spatshift_across_scale_crosscorrs']
Final size of features concatenated is [62 x 641]
Running PCA on higher-level features...
Preparing for PCA: original dims of features:
(62, 592)
Applying pre-computed PCA matrix...
Final size of features concatenated is [62 x 51]
Preparing for PCA: original dims of features:
(62, 151)
Applying pre-computed PCA matrix...
Final size of feature matrix is:
(62, 1)


In [16]:
defined.shape

(600,)

In [28]:
_feature_extractor.modules[0].max_features

449

In [29]:
_feature_extractor.modules[1].max_features

151

In [17]:
pc = []
for m in _feature_extractor.modules:           
    if hasattr(m, 'pct_var_expl'):
        pcm = [m.pct_var_expl, m.min_pct_var,  m.n_comp_needed]                  
    else:
        pcm = None
    pc.append(pcm)

In [23]:
pc[0][0]

array([[ 0.        ,  0.        , 98.06194   , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.7108457 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.49869698, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [17]:
torch.cat((feats, feats2), axis=1)

tensor([[ 4.1239e-01,  4.9881e-03,  3.6118e-02,  ..., -1.7653e+02,
         -1.7898e+01, -7.5757e-05],
        [ 3.7503e-02,  6.5405e-03,  2.0029e+00,  ..., -1.8690e+02,
         -1.1006e+01,  1.4115e-04],
        [ 2.0118e-01,  4.4240e-03,  1.1431e+00,  ..., -1.7151e+02,
         -1.0463e+01,  2.8603e-05],
        ...,
        [ 1.5854e-02,  5.0241e-05,  1.2342e+00,  ...,  1.2241e+03,
          6.5767e+00, -3.6620e-04],
        [ 2.7447e-01,  1.8736e-02,  1.6470e+00,  ..., -2.3180e+01,
         -1.4085e+01,  3.3083e-04],
        [ 5.7150e-02,  1.9648e-05,  2.8961e+00,  ..., -1.9614e+02,
         -1.2480e+01, -4.2701e-04]])

In [38]:
feats, defined = _feature_extractor(val_stim_data, models[mm], mm, fitting_mode=False)

Index into batch for prf 2: 2
Size of features array for this image set and prf is:
(62, 641)
Final size of features concatenated is [62 x 641]
Feature types included are:
['pixel_stats', 'mean_magnitudes', 'mean_realparts', 'marginal_stats_lowpass_recons', 'variance_highpass_resid', 'magnitude_feature_autocorrs', 'lowpass_recon_autocorrs', 'highpass_resid_autocorrs', 'magnitude_within_scale_crosscorrs', 'real_within_scale_crosscorrs', 'magnitude_across_scale_crosscorrs', 'real_imag_across_scale_crosscorrs', 'real_spatshift_within_scale_crosscorrs', 'real_spatshift_across_scale_crosscorrs']
Final size of features concatenated is [62 x 641]
Running PCA on higher-level features...
Preparing for PCA: original dims of features:
(62, 592)
Applying pre-computed PCA matrix...
Final size of features concatenated is [62 x 51]


In [27]:
_feature_extractor.modules[0].get_partial_versions()[0].shape

(3, 641)

In [36]:
np.where(defined==False)[0][0]

51

In [64]:
# _feature_extractor.n_comp_needed

In [66]:
hasattr(_feature_extractor, 'pct_kdfvar_expl')

False